In [ ]:
IMAGE_FOLDER = r"C:\Users\samih\Documents\ShareX\Screenshots\2025-03\33x33 - 1sec\v1 - 57"
images_folder = IMAGE_FOLDER

# RAW Image files

### Animation of loaded in RAW Images

In [ ]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, display

# Folder where your images are stored (adjust the path accordingly)
images_folder = IMAGE_FOLDER

# Get sorted list of image file paths sorted by modification time (oldest first)
image_paths = sorted(glob.glob(os.path.join(images_folder, '*.png')), key=os.path.getmtime)
if not image_paths:
    image_paths = sorted(glob.glob(os.path.join(images_folder, '*.jpg')), key=os.path.getmtime)

# Optionally, if you want the newest files first, reverse the list:
# image_paths.reverse()

# Load images using OpenCV and convert from BGR to RGB
images = []
for path in image_paths:
    img = cv2.imread(path)
    if img is None:
        print(f"Warning: Could not load image {path}")
    else:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        images.append(img_rgb)

if len(images) == 0:
    raise ValueError("No images were loaded. Please check your folder path and image file types.")

# Set up the plot
fig, ax = plt.subplots()
im_plot = ax.imshow(images[0])
title = ax.set_title("Image 1")
ax.axis('off')

def update(frame):
    """Update function for the animation."""
    im_plot.set_data(images[frame])
    title.set_text(f"Image {frame + 1}")
    return im_plot, title

# Create the animation and assign to a persistent variable 'anim'
anim = animation.FuncAnimation(fig, update, frames=len(images), interval=500, blit=False, repeat=True)

# Display the animation as HTML (for Jupyter environments)
display(HTML(anim.to_jshtml()))

plt.show()


# Computer Vision

### Creating a new clearer discretised version - Animation

In [ ]:
import os
import glob
import re
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, display

# ---------------------------------------------------
# Helper function: classify a cell's average color
# ---------------------------------------------------
def classify_cell_color(avg_bgr):
    """
    Classify a cell color based on the average BGR value.
    Returns one of: 'green', 'orange', or 'white'.
    (We will handle 'black' explicitly for the outer border.)
    
    NOTE: You will likely need to adjust these thresholds to
    match your images more accurately.
    """
    b, g, r = avg_bgr
    
    # Convert BGR -> HSV (often easier for color-based thresholds)
    color_bgr = np.uint8([[avg_bgr]])  # shape (1,1,3)
    color_hsv = cv2.cvtColor(color_bgr.reshape(1,1,3), cv2.COLOR_BGR2HSV).reshape(3,)
    h, s, v = color_hsv
    
    # Example thresholds (very rough):
    # - green (leaves): hue ~ 30..90, s>50, v>50
    # - orange (fire): hue ~ 5..25, s>100, v>100
    # - white (burnt/empty): fallback if not green/orange
    if 30 <= h <= 90 and s > 50 and v > 50:
        return 'green'
    # elif 5 <= h <= 25 and s > 100 and v > 100:
    elif 0 <= h <= 40 and s > 60 and v > 60:
        return 'orange'
    else:
        return 'white'

# ---------------------------------------------------
# Function to process one image into a 33x33 grid
# ---------------------------------------------------
def process_image_into_grid(img, grid_size=33):
    """
    1. Split the image into a grid_size x grid_size cells.
    2. For each cell, compute the average color (BGR).
    3. Classify outer border as black, and inner cells as green/orange/white.
    4. Return a 2D array of labels ('black', 'green', 'orange', 'white')
       and a color-coded 33x33 image in RGB.
    """
    height, width, _ = img.shape
    
    # Calculate cell size in pixels
    cell_h = height // grid_size
    cell_w = width // grid_size
    
    # We'll store labels in a 2D list
    labels = []
    
    # We'll build a new 33x33 image (1 pixel per cell) for visualization
    # in RGB. Alternatively, you could scale up each cell to 10x10, etc.
    classified_img = np.zeros((grid_size, grid_size, 3), dtype=np.uint8)
    
    for row in range(grid_size):
        row_labels = []
        for col in range(grid_size):
            # Outer border forced to 'black'
            if row == 0 or row == grid_size - 1 or col == 0 or col == grid_size - 1:
                label = 'black'
            else:
                # Compute the average color in the cell
                y_start = row * cell_h
                y_end = (row + 1) * cell_h
                x_start = col * cell_w
                x_end = (col + 1) * cell_w
                
                cell = img[y_start:y_end, x_start:x_end]
                avg_b = np.mean(cell[:,:,0])
                avg_g = np.mean(cell[:,:,1])
                avg_r = np.mean(cell[:,:,2])
                
                # Classify the cell
                label = classify_cell_color((avg_b, avg_g, avg_r))
            
            row_labels.append(label)
            
            # Convert label -> color (RGB)
            if label == 'black':
                classified_img[row, col] = (0, 0, 0)
            elif label == 'green':
                classified_img[row, col] = (0, 255, 0)
            elif label == 'orange':
                classified_img[row, col] = (255, 165, 0)
            else:  # 'white'
                classified_img[row, col] = (255, 255, 255)
        
        labels.append(row_labels)
    
    return np.array(labels), classified_img

# ---------------------------------------------------
# Main workflow
# ---------------------------------------------------
def main_classification_workflow(images_folder):
    """
    1. Collect all images from a folder (in date/time or name order).
    2. For each image:
       - read with OpenCV
       - discretize into 33x33 grid
       - produce a color-coded classification image
    3. Animate the resulting classification images in order.
    """
    # Adjust how you want to sort:
    #   by modification time (oldest first) OR by name
    image_paths = sorted(glob.glob(os.path.join(images_folder, '*.png')), key=os.path.getmtime)
    if not image_paths:
        image_paths = sorted(glob.glob(os.path.join(images_folder, '*.jpg')), key=os.path.getmtime)
    
    if not image_paths:
        raise ValueError("No images found in the specified folder.")
    
    classified_images = []  # store the color-coded classification images
    labels_over_time = []    # store the label grids if you need them
    
    for path in image_paths:
        img = cv2.imread(path)
        if img is None:
            print(f"Warning: Could not load image {path}")
            continue
        
        # Convert from BGR to RGB for consistency with matplotlib
        # (But we still do color classification in BGR, which is fine.)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        labels_grid, classified_img = process_image_into_grid(img, grid_size=33)
        
        labels_over_time.append(labels_grid)
        classified_images.append(classified_img)
    
    # Once we have all the classification images, let's animate them
    animate_classified_images(classified_images)

def animate_classified_images(images_list):
    """
    Given a list of color-coded images (all 33x33 in shape),
    create a Matplotlib animation and display inline (Jupyter).
    """
    fig, ax = plt.subplots()
    # Show the first image
    im_plot = ax.imshow(images_list[0])
    ax.set_title("Classified Image 1")
    ax.axis('off')
    
    def update(frame):
        im_plot.set_data(images_list[frame])
        ax.set_title(f"Classified Image {frame + 1}")
        return [im_plot]
    
    anim = animation.FuncAnimation(
        fig, update, frames=len(images_list),
        interval=500, blit=False, repeat=True
    )
    
    # Display inline if in a Jupyter environment
    display(HTML(anim.to_jshtml()))
    plt.show()

# ---------------------------------------------------
# Example usage
# ---------------------------------------------------
if __name__ == "__main__":
    # Replace with your folder containing the images
    images_folder = IMAGE_FOLDER
    main_classification_workflow(images_folder)


### Creating a new clearer discretised version - Side by Side


In [ ]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

# ---------------------------------------------------
# Helper function: classify a cell's average color
# ---------------------------------------------------
def classify_cell_color(avg_bgr):
    """
    Classify a cell color based on the average BGR value.
    Returns one of: 'green', 'orange', or 'white'.
    (The outer border will be forced to 'black' elsewhere.)
    """
    b, g, r = avg_bgr

    # Convert BGR -> HSV (often easier for color thresholds)
    color_bgr = np.uint8([[avg_bgr]])  # shape (1,1,3)
    color_hsv = cv2.cvtColor(color_bgr.reshape(1,1,3), cv2.COLOR_BGR2HSV).reshape(3,)
    h, s, v = color_hsv

    # Example thresholds (adjust as needed):
    if 30 <= h <= 90 and s > 50 and v > 50:
        return 'green'
    # Broader orange range:
    elif 0 <= h <= 40 and s > 60 and v > 60:
        return 'orange'
    else:
        return 'white'

# ---------------------------------------------------
# Function to process one image into a 33x33 grid
# ---------------------------------------------------
def process_image_into_grid(img, grid_size=33):
    """
    Splits the image into grid_size x grid_size cells.
    For each cell, computes the average BGR color and
    classifies the cell:
      - Outer border cells are forced to 'black'
      - Inner cells are classified as 'green', 'orange', or 'white'
    Returns:
      - A 2D array of labels (shape: grid_size x grid_size)
      - A color-coded classification image (in RGB, one pixel per cell)
    """
    height, width, _ = img.shape
    cell_h = height // grid_size
    cell_w = width // grid_size

    labels = []
    classified_img = np.zeros((grid_size, grid_size, 3), dtype=np.uint8)

    for row in range(grid_size):
        row_labels = []
        for col in range(grid_size):
            # Force outer border cells to 'black'
            if row == 0 or row == grid_size - 1 or col == 0 or col == grid_size - 1:
                label = 'black'
            else:
                y_start = row * cell_h
                y_end = (row + 1) * cell_h
                x_start = col * cell_w
                x_end = (col + 1) * cell_w

                cell = img[y_start:y_end, x_start:x_end]
                avg_b = np.mean(cell[:, :, 0])
                avg_g = np.mean(cell[:, :, 1])
                avg_r = np.mean(cell[:, :, 2])
                label = classify_cell_color((avg_b, avg_g, avg_r))

            row_labels.append(label)

            # Convert the label into an RGB color
            if label == 'black':
                classified_img[row, col] = (0, 0, 0)
            elif label == 'green':
                classified_img[row, col] = (0, 255, 0)
            elif label == 'orange':
                classified_img[row, col] = (255, 165, 0)
            else:  # 'white'
                classified_img[row, col] = (255, 255, 255)

        labels.append(row_labels)

    return np.array(labels), classified_img

# ---------------------------------------------------
# Function to display each original image alongside its processed version
# in separate figures (one figure per image).
# ---------------------------------------------------
def show_comparisons_per_image(images_folder):
    """
    For each image in the folder, load the original image and
    create its classification image. Then, display them side by side
    in a separate figure for each image.
    """
    # Get image paths sorted by modification time
    image_paths = sorted(glob.glob(os.path.join(images_folder, '*.png')), key=os.path.getmtime)
    if not image_paths:
        image_paths = sorted(glob.glob(os.path.join(images_folder, '*.jpg')), key=os.path.getmtime)
    if not image_paths:
        raise ValueError("No images found in the specified folder.")

    for idx, path in enumerate(image_paths):
        # Read the original image (BGR) and convert to RGB for display
        orig = cv2.imread(path)
        if orig is None:
            print(f"Warning: Could not load image {path}")
            continue
        orig_rgb = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)

        # Process the image to get the classification image (33x33)
        _, classified_img = process_image_into_grid(orig, grid_size=33)

        # Upscale the classified image for better visualization (e.g., 10x)
        upscale_factor = 10
        upscaled_classified = cv2.resize(
            classified_img,
            (classified_img.shape[1] * upscale_factor, classified_img.shape[0] * upscale_factor),
            interpolation=cv2.INTER_NEAREST
        )

        # Create a new figure for each pair
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        
        # Left: Original image
        axes[0].imshow(orig_rgb)
        axes[0].set_title(f"Original Image {idx + 1}")
        axes[0].axis('off')

        # Right: Processed classification image
        axes[1].imshow(upscaled_classified)
        axes[1].set_title(f"Classified Image {idx + 1}")
        axes[1].axis('off')

        plt.tight_layout()
        plt.show()

# ---------------------------------------------------
# Example usage
# ---------------------------------------------------
if __name__ == "__main__":
    # Replace with the folder containing your screenshots
    images_folder = IMAGE_FOLDER
    show_comparisons_per_image(images_folder)


### Quantifying the spread by counting fire blocks per image 

In [ ]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

# TIME STEP BETWEEN IMAGES = 1
TIME_STEP = 1  # seconds

def classify_cell_color(avg_bgr):
    # Same logic as before; broaden orange if needed
    b, g, r = avg_bgr
    color_bgr = np.uint8([[avg_bgr]])
    color_hsv = cv2.cvtColor(color_bgr.reshape(1,1,3), cv2.COLOR_BGR2HSV).reshape(3,)
    h, s, v = color_hsv
    
    if 30 <= h <= 90 and s > 50 and v > 50:
        return 'green'
    elif 0 <= h <= 40 and s > 60 and v > 60:
        return 'orange'
    else:
        return 'white'

def process_image_into_grid(img, grid_size=33):
    height, width, _ = img.shape
    cell_h = height // grid_size
    cell_w = width // grid_size
    
    labels_grid = np.empty((grid_size, grid_size), dtype=object)
    
    for row in range(grid_size):
        for col in range(grid_size):
            # Force outer border cells to 'black'
            if row == 0 or row == grid_size - 1 or col == 0 or col == grid_size - 1:
                labels_grid[row, col] = 'black'
            else:
                y_start = row * cell_h
                y_end   = (row + 1) * cell_h
                x_start = col * cell_w
                x_end   = (col + 1) * cell_w
                
                cell = img[y_start:y_end, x_start:x_end]
                avg_b = np.mean(cell[:,:,0])
                avg_g = np.mean(cell[:,:,1])
                avg_r = np.mean(cell[:,:,2])
                labels_grid[row, col] = classify_cell_color((avg_b, avg_g, avg_r))
    
    return labels_grid

def load_and_classify_images(images_folder, grid_size=33):
    """
    Loads all images from the folder, sorts by modification time,
    returns a list of classification grids (each is 33x33 with labels).
    """
    image_paths = sorted(glob.glob(os.path.join(images_folder, '*.png')), key=os.path.getmtime)
    if not image_paths:
        image_paths = sorted(glob.glob(os.path.join(images_folder, '*.jpg')), key=os.path.getmtime)
    if not image_paths:
        raise ValueError("No images found in the specified folder.")
    
    classification_grids = []
    for path in image_paths:
        img = cv2.imread(path)
        if img is None:
            print(f"Warning: Could not load image {path}")
            continue
        classification_grid = process_image_into_grid(img, grid_size=grid_size)
        classification_grids.append(classification_grid)
    
    return classification_grids

def calculate_fire_spread(classification_grids, time_step=TIME_STEP):
    """
    Calculates a simple spread metric: 
    - For each grid, count how many cells are labeled 'orange'.
    - Then compute the discrete difference in orange count over time.
    Returns:
    - orange_counts: list of total 'orange' cells per time step
    - spread_rates: list of discrete rate of spread (d(orange_count)/dt)
    """
    orange_counts = []
    for grid in classification_grids:
        count_orange = np.sum(grid == 'orange')
        orange_counts.append(count_orange)
    
    spread_rates = []
    for i in range(1, len(orange_counts)):
        diff = orange_counts[i] - orange_counts[i-1]
        # Rate = difference in count / time_step
        spread_rates.append(diff / time_step)
    
    return orange_counts, spread_rates

def main(images_folder):
    # 1. Load and classify all images into 33x33 grids
    classification_grids = load_and_classify_images(images_folder, grid_size=33)
    
    # 2. Calculate a simple fire spread metric
    orange_counts, spread_rates = calculate_fire_spread(classification_grids, time_step=TIME_STEP)
    
    # 3. Print or plot the results
    # Print them for demonstration:
    print("Orange cell counts over time:", orange_counts)
    print("Spread rates (cells per second) between consecutive images:", spread_rates)
    
    # Optionally, plot
    time_steps = [i * TIME_STEP for i in range(len(orange_counts))]
    
    plt.figure(figsize=(8,5))
    plt.plot(time_steps, orange_counts, marker='o', label='Orange Cell Count')
    plt.xlabel("Time (seconds)")
    plt.ylabel("Number of Orange Cells")
    plt.title("Fire Spread Over Time (Orange Cell Count)")
    plt.legend()
    plt.show()
    
    # Spread rate plot (use midpoints between images for x-axis if you like)
    if len(spread_rates) > 0:
        # midpoints in time
        rate_times = [(time_steps[i] + time_steps[i+1]) / 2 for i in range(len(time_steps)-1)]
        plt.figure(figsize=(8,5))
        plt.plot(rate_times, spread_rates, marker='x', color='red', label='Spread Rate')
        plt.xlabel("Time (seconds)")
        plt.ylabel("Cells per second")
        plt.title("Rate of Spread")
        plt.legend()
        plt.show()

if __name__ == "__main__":
    # TIME STEP BETWEEN IMAGES = 1
    images_folder = IMAGE_FOLDER
    main(images_folder)


### Calculates it as a radius?? - Need to check the maths here

In [ ]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

# TIME STEP BETWEEN IMAGES = 1
TIME_STEP = 1  # seconds

def classify_cell_color(avg_bgr):
    """
    Classify a cell color based on the average BGR value.
    Returns one of: 'green', 'orange', or 'white'.
    (The outer border will be forced to 'black' elsewhere.)
    """
    b, g, r = avg_bgr
    color_bgr = np.uint8([[avg_bgr]])
    color_hsv = cv2.cvtColor(color_bgr.reshape(1,1,3), cv2.COLOR_BGR2HSV).reshape(3,)
    h, s, v = color_hsv

    # Green threshold
    if 30 <= h <= 90 and s > 50 and v > 50:
        return 'green'
    # Broadened orange threshold for fire
    elif 0 <= h <= 40 and s > 60 and v > 60:
        return 'orange'
    else:
        return 'white'

def process_image_into_grid(img, grid_size=33):
    """
    Splits the image into grid_size x grid_size cells.
    Returns a 2D array (grid) of labels.
    Outer border cells are forced to 'black'; inner cells are classified.
    """
    height, width, _ = img.shape
    cell_h = height // grid_size
    cell_w = width // grid_size
    labels_grid = np.empty((grid_size, grid_size), dtype=object)

    for row in range(grid_size):
        for col in range(grid_size):
            if row == 0 or row == grid_size - 1 or col == 0 or col == grid_size - 1:
                labels_grid[row, col] = 'black'
            else:
                y_start = row * cell_h
                y_end = (row + 1) * cell_h
                x_start = col * cell_w
                x_end = (col + 1) * cell_w
                cell = img[y_start:y_end, x_start:x_end]
                avg_b = np.mean(cell[:,:,0])
                avg_g = np.mean(cell[:,:,1])
                avg_r = np.mean(cell[:,:,2])
                labels_grid[row, col] = classify_cell_color((avg_b, avg_g, avg_r))
    return labels_grid

def load_classification_grids(images_folder, grid_size=33):
    """
    Loads all images from the folder (sorted by modification time)
    and returns a list of classification grids.
    """
    image_paths = sorted(glob.glob(os.path.join(images_folder, '*.png')), key=os.path.getmtime)
    if not image_paths:
        image_paths = sorted(glob.glob(os.path.join(images_folder, '*.jpg')), key=os.path.getmtime)
    if not image_paths:
        raise ValueError("No images found in the specified folder.")
    
    grids = []
    for path in image_paths:
        img = cv2.imread(path)
        if img is None:
            print(f"Warning: Could not load image {path}")
            continue
        grid = process_image_into_grid(img, grid_size=grid_size)
        grids.append(grid)
    return grids

def calculate_fire_radius(classification_grid):
    """
    Calculates the fire radius as the maximum Euclidean distance (in grid cells)
    from the center of the grid to any cell labeled as 'orange'.
    """
    grid_size = classification_grid.shape[0]
    center = np.array([grid_size/2, grid_size/2])
    # Get indices of orange cells
    orange_cells = np.argwhere(classification_grid == 'orange')
    if orange_cells.size == 0:
        return 0
    distances = np.linalg.norm(orange_cells - center, axis=1)
    return np.max(distances)

def calculate_rate_of_spread(grids, time_step=TIME_STEP):
    """
    Calculates the rate of spread based on the change in fire radius.
    Returns:
      - radii: list of fire radii (in grid cells) at each time step.
      - rates: list of rate of spread (grid cells per second) computed between time steps.
    """
    radii = [calculate_fire_radius(grid) for grid in grids]
    rates = []
    for i in range(1, len(radii)):
        rate = (radii[i] - radii[i-1]) / time_step
        rates.append(rate)
    return radii, rates

def main(images_folder):
    grids = load_classification_grids(images_folder, grid_size=33)
    radii, rates = calculate_rate_of_spread(grids, time_step=TIME_STEP)
    
    # Print out the computed radii and rate of spread
    print("Fire radii (in grid cells):", radii)
    print("Rate of spread (grid cells per second):", rates)
    
    # Plot fire radii over time
    time_points = [i * TIME_STEP for i in range(len(radii))]
    plt.figure(figsize=(8,5))
    plt.plot(time_points, radii, marker='o', label='Fire Radius')
    plt.xlabel("Time (seconds)")
    plt.ylabel("Fire Radius (grid cells)")
    plt.title("Fire Spread Over Time")
    plt.legend()
    plt.show()
    
    # Plot rate of spread
    if rates:
        # Use midpoints for time axis for the rate plot
        rate_times = [(time_points[i] + time_points[i+1]) / 2 for i in range(len(time_points)-1)]
        plt.figure(figsize=(8,5))
        plt.plot(rate_times, rates, marker='x', color='red', label='Rate of Spread')
        plt.xlabel("Time (seconds)")
        plt.ylabel("Spread Rate (grid cells per second)")
        plt.title("Rate of Spread")
        plt.legend()
        plt.show()

if __name__ == "__main__":
    # TIME STEP BETWEEN IMAGES = 1
    images_folder = IMAGE_FOLDER
    main(images_folder)


# Isochrones

### Outline Isochrone (Blocky) - check with image side by side

In [ ]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

# ---------------------------------------------------
# Helper function: classify a cell's average color
# ---------------------------------------------------
def classify_cell_color(avg_bgr):
    """
    Classify a cell color based on the average BGR value.
    Returns one of: 'green', 'orange', or 'white'.
    (The outer border will be forced to 'black' elsewhere.)
    """
    b, g, r = avg_bgr
    color_bgr = np.uint8([[avg_bgr]])
    color_hsv = cv2.cvtColor(color_bgr.reshape(1,1,3), cv2.COLOR_BGR2HSV).reshape(3,)
    h, s, v = color_hsv

    # Adjust thresholds as needed
    if 30 <= h <= 90 and s > 50 and v > 50:
        return 'green'
    elif 0 <= h <= 40 and s > 60 and v > 60:
        return 'orange'
    else:
        return 'white'

# ---------------------------------------------------
# Function to process one image into a 33x33 grid
# ---------------------------------------------------
def process_image_into_grid(img, grid_size=33):
    """
    Splits the image into grid_size x grid_size cells.
    For each cell, computes the average BGR color and
    classifies the cell:
      - Outer border cells are forced to 'black'
      - Inner cells are classified as 'green', 'orange', or 'white'
    Returns:
      - A 2D array of labels (shape: grid_size x grid_size)
      - A color-coded classification image (in RGB, one pixel per cell)
    """
    height, width, _ = img.shape
    cell_h = height // grid_size
    cell_w = width // grid_size

    labels = []
    classified_img = np.zeros((grid_size, grid_size, 3), dtype=np.uint8)

    for row in range(grid_size):
        row_labels = []
        for col in range(grid_size):
            # Force outer border cells to 'black'
            if row == 0 or row == grid_size - 1 or col == 0 or col == grid_size - 1:
                label = 'black'
            else:
                y_start = row * cell_h
                y_end   = (row + 1) * cell_h
                x_start = col * cell_w
                x_end   = (col + 1) * cell_w
                cell = img[y_start:y_end, x_start:x_end]
                avg_b = np.mean(cell[:,:,0])
                avg_g = np.mean(cell[:,:,1])
                avg_r = np.mean(cell[:,:,2])
                label = classify_cell_color((avg_b, avg_g, avg_r))

            row_labels.append(label)

            # Convert label -> RGB color
            if label == 'black':
                classified_img[row, col] = (0, 0, 0)
            elif label == 'green':
                classified_img[row, col] = (0, 255, 0)
            elif label == 'orange':
                classified_img[row, col] = (255, 165, 0)
            else:  # 'white'
                classified_img[row, col] = (255, 255, 255)

        labels.append(row_labels)

    return np.array(labels), classified_img

# ---------------------------------------------------
# Function to draw the firefront outline
# ---------------------------------------------------
def draw_firefront_outline(labels_grid, upscaled_classified, upscale_factor=10):
    """
    1. Create a binary mask for cells that are either 'orange' or 'white'.
    2. Upscale the mask to match the upscaled_classified size.
    3. Find contours and draw them in red on a copy of upscaled_classified.
    """
    grid_size = labels_grid.shape[0]
    # Create a 2D mask: mark cells 'orange' or 'white' as 255
    mask = np.zeros((grid_size, grid_size), dtype=np.uint8)
    for r in range(grid_size):
        for c in range(grid_size):
            if labels_grid[r, c] in ('orange', 'white'):
                mask[r, c] = 255

    # Upscale mask
    upscaled_size = (grid_size * upscale_factor, grid_size * upscale_factor)
    mask_upscaled = cv2.resize(mask, upscaled_size, interpolation=cv2.INTER_NEAREST)

    # Find contours on the upscaled mask
    contours, _ = cv2.findContours(mask_upscaled, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return upscaled_classified  # no fire reached cells at all

    # Draw all contours in red on a copy of upscaled_classified
    overlay = upscaled_classified.copy()
    cv2.drawContours(overlay, contours, -1, (255, 0, 0), 2)  # Red color in BGR (255, 0, 0)
    return overlay


# ---------------------------------------------------
# Main function to show each classified image with a firefront outline
# ---------------------------------------------------
def show_firefront_outlines(images_folder):
    """
    For each image in the folder:
      1) Classify the image into a 33x33 grid.
      2) Upscale the classified image.
      3) Draw the firefront outline (orange cells) on the upscaled image.
      4) Show them side by side.
    """
    image_paths = sorted(glob.glob(os.path.join(images_folder, '*.png')), key=os.path.getmtime)
    if not image_paths:
        image_paths = sorted(glob.glob(os.path.join(images_folder, '*.jpg')), key=os.path.getmtime)
    if not image_paths:
        raise ValueError("No images found in the specified folder.")

    upscale_factor = 10
    grid_size = 33

    for idx, path in enumerate(image_paths):
        img_bgr = cv2.imread(path)
        if img_bgr is None:
            print(f"Warning: Could not load image {path}")
            continue

        # Classify the image
        labels_grid, classified_img = process_image_into_grid(img_bgr, grid_size=grid_size)

        # Upscale the classified image
        upscaled_classified = cv2.resize(
            classified_img,
            (classified_img.shape[1] * upscale_factor, classified_img.shape[0] * upscale_factor),
            interpolation=cv2.INTER_NEAREST
        )

        # Draw the firefront outline
        outlined_img = draw_firefront_outline(labels_grid, upscaled_classified, upscale_factor=upscale_factor)

        # Display side by side
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        axes[0].imshow(upscaled_classified)
        axes[0].set_title(f"Classified Image {idx + 1}")
        axes[0].axis('off')

        axes[1].imshow(outlined_img)
        axes[1].set_title(f"Firefront Outline {idx + 1}")
        axes[1].axis('off')

        plt.tight_layout()
        plt.show()

# ---------------------------------------------------
# Example usage
# ---------------------------------------------------
if __name__ == "__main__":
    images_folder = IMAGE_FOLDER
    show_firefront_outlines(images_folder)


### Isochrone Images individual

In [ ]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

def classify_cell_color(avg_bgr):
    """
    Classify a cell color based on the average BGR value.
    Returns one of: 'green', 'orange', or 'white'.
    (The outer border will be forced to 'black'.)
    """
    b, g, r = avg_bgr
    color_bgr = np.uint8([[avg_bgr]])
    color_hsv = cv2.cvtColor(color_bgr.reshape(1,1,3), cv2.COLOR_BGR2HSV).reshape(3,)
    h, s, v = color_hsv

    if 30 <= h <= 90 and s > 50 and v > 50:
        return 'green'
    elif 0 <= h <= 40 and s > 60 and v > 60:
        return 'orange'
    else:
        return 'white'

def process_image_into_grid(img, grid_size=33):
    """
    Splits the image into grid_size x grid_size cells.
    For each cell, computes the average BGR color and
    classifies the cell:
      - Outer border cells are forced to 'black'
      - Inner cells are classified as 'green', 'orange', or 'white'
    Returns:
      - A 2D array of labels (shape: grid_size x grid_size)
      - A color-coded classification image (RGB, one pixel per cell)
    """
    height, width, _ = img.shape
    cell_h = height // grid_size
    cell_w = width // grid_size

    labels = []
    classified_img = np.zeros((grid_size, grid_size, 3), dtype=np.uint8)

    for row in range(grid_size):
        row_labels = []
        for col in range(grid_size):
            if row == 0 or row == grid_size - 1 or col == 0 or col == grid_size - 1:
                label = 'black'
            else:
                y_start = row * cell_h
                y_end   = (row + 1) * cell_h
                x_start = col * cell_w
                x_end   = (col + 1) * cell_w
                cell = img[y_start:y_end, x_start:x_end]
                avg_b = np.mean(cell[:,:,0])
                avg_g = np.mean(cell[:,:,1])
                avg_r = np.mean(cell[:,:,2])
                label = classify_cell_color((avg_b, avg_g, avg_r))
            row_labels.append(label)
            if label == 'black':
                classified_img[row, col] = (0, 0, 0)
            elif label == 'green':
                classified_img[row, col] = (0, 255, 0)
            elif label == 'orange':
                classified_img[row, col] = (255, 165, 0)
            else:
                classified_img[row, col] = (255, 255, 255)
        labels.append(row_labels)

    return np.array(labels), classified_img

def draw_isochrone_only(labels_grid, upscale_factor=10):
    """
    Creates a binary mask from the classification grid for cells labeled as
    'orange' or 'white', upscales it, finds the contours, and then draws only
    the outline (isochrone) on a blank white background.
    """
    grid_size = labels_grid.shape[0]
    mask = np.zeros((grid_size, grid_size), dtype=np.uint8)
    for r in range(grid_size):
        for c in range(grid_size):
            if labels_grid[r, c] in ('orange', 'white'):
                mask[r, c] = 255

    # Upscale the mask to the final size.
    upscaled_size = (grid_size * upscale_factor, grid_size * upscale_factor)
    mask_upscaled = cv2.resize(mask, upscaled_size, interpolation=cv2.INTER_NEAREST)

    # Find contours.
    contours, _ = cv2.findContours(mask_upscaled, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create a blank white image.
    outline_img = 255 * np.ones((upscaled_size[1], upscaled_size[0], 3), dtype=np.uint8)

    # Draw all contours in red.
    if contours:
        cv2.drawContours(outline_img, contours, -1, (255, 0, 0), 2)  # Red color in BGR.
    return outline_img

def show_isochrone_only(images_folder):
    """
    For each image in the folder, classify the image into a 33x33 grid,
    then create an image that shows only the isochrone (outline of cells labeled as
    'orange' or 'white') on a blank white background.
    """
    image_paths = sorted(glob.glob(os.path.join(images_folder, '*.png')), key=os.path.getmtime)
    if not image_paths:
        image_paths = sorted(glob.glob(os.path.join(images_folder, '*.jpg')), key=os.path.getmtime)
    if not image_paths:
        raise ValueError("No images found in the specified folder.")

    upscale_factor = 10
    grid_size = 33

    for idx, path in enumerate(image_paths):
        img = cv2.imread(path)
        if img is None:
            print(f"Warning: Could not load image {path}")
            continue

        # Process the image to get the classification grid.
        labels_grid, _ = process_image_into_grid(img, grid_size=grid_size)

        # Create the isochrone-only image.
        isochrone_img = draw_isochrone_only(labels_grid, upscale_factor=upscale_factor)

        # Display the isochrone-only image.
        plt.figure(figsize=(6,6))
        plt.imshow(cv2.cvtColor(isochrone_img, cv2.COLOR_BGR2RGB))
        plt.title(f"Isochrone Only {idx + 1}")
        plt.axis('off')
        plt.tight_layout()
        plt.show()

if __name__ == "__main__":
    images_folder = IMAGE_FOLDER
    show_isochrone_only(images_folder)


### Overlay of Isochrones (too busy and messy same colour)

In [ ]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

def classify_cell_color(avg_bgr):
    """
    Classify a cell color based on the average BGR value.
    Returns one of: 'green', 'orange', or 'white'.
    (The outer border is forced to 'black'.)
    """
    b, g, r = avg_bgr
    color_bgr = np.uint8([[avg_bgr]])
    color_hsv = cv2.cvtColor(color_bgr.reshape(1,1,3), cv2.COLOR_BGR2HSV).reshape(3,)
    h, s, v = color_hsv

    if 30 <= h <= 90 and s > 50 and v > 50:
        return 'green'
    elif 0 <= h <= 40 and s > 60 and v > 60:
        return 'orange'
    else:
        return 'white'

def process_image_into_grid(img, grid_size=33):
    """
    Splits the image into grid_size x grid_size cells.
    For each cell, computes the average BGR color and
    classifies the cell:
      - Outer border cells are forced to 'black'
      - Inner cells are classified as 'green', 'orange', or 'white'
    Returns:
      - A 2D array of labels (shape: grid_size x grid_size)
      - A color-coded classification image (RGB, one pixel per cell)
    """
    height, width, _ = img.shape
    cell_h = height // grid_size
    cell_w = width // grid_size

    labels = []
    classified_img = np.zeros((grid_size, grid_size, 3), dtype=np.uint8)

    for row in range(grid_size):
        row_labels = []
        for col in range(grid_size):
            if row == 0 or row == grid_size - 1 or col == 0 or col == grid_size - 1:
                label = 'black'
            else:
                y_start = row * cell_h
                y_end   = (row + 1) * cell_h
                x_start = col * cell_w
                x_end   = (col + 1) * cell_w
                cell = img[y_start:y_end, x_start:x_end]
                avg_b = np.mean(cell[:,:,0])
                avg_g = np.mean(cell[:,:,1])
                avg_r = np.mean(cell[:,:,2])
                label = classify_cell_color((avg_b, avg_g, avg_r))
            row_labels.append(label)
            if label == 'black':
                classified_img[row, col] = (0, 0, 0)
            elif label == 'green':
                classified_img[row, col] = (0, 255, 0)
            elif label == 'orange':
                classified_img[row, col] = (255, 165, 0)
            else:
                classified_img[row, col] = (255, 255, 255)
        labels.append(row_labels)

    return np.array(labels), classified_img

def get_isochrone_contours(labels_grid, upscale_factor=10):
    """
    Creates a binary mask from the classification grid for cells labeled as
    'orange' or 'white', upscales it, and finds contours (firefront outlines).
    """
    grid_size = labels_grid.shape[0]
    mask = np.zeros((grid_size, grid_size), dtype=np.uint8)
    for r in range(grid_size):
        for c in range(grid_size):
            if labels_grid[r, c] in ('orange', 'white'):
                mask[r, c] = 255

    upscaled_size = (grid_size * upscale_factor, grid_size * upscale_factor)
    mask_upscaled = cv2.resize(mask, upscaled_size, interpolation=cv2.INTER_NEAREST)
    contours, _ = cv2.findContours(mask_upscaled, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def overlay_all_isochrones(images_folder, grid_size=33, upscale_factor=10, line_thickness=1):
    """
    For all images in the folder, compute the firefront contours and overlay them
    on a single blank white canvas.
    
    The outlines are drawn with a thin line (set by line_thickness) in red.
    """
    # Get image paths sorted by modification time
    image_paths = sorted(glob.glob(os.path.join(images_folder, '*.png')), key=os.path.getmtime)
    if not image_paths:
        image_paths = sorted(glob.glob(os.path.join(images_folder, '*.jpg')), key=os.path.getmtime)
    if not image_paths:
        raise ValueError("No images found in the specified folder.")

    upscaled_size = (grid_size * upscale_factor, grid_size * upscale_factor)
    overlay = 255 * np.ones((upscaled_size[1], upscaled_size[0], 3), dtype=np.uint8)  # white canvas

    for path in image_paths:
        img = cv2.imread(path)
        if img is None:
            print(f"Warning: Could not load image {path}")
            continue
        labels_grid, _ = process_image_into_grid(img, grid_size=grid_size)
        contours = get_isochrone_contours(labels_grid, upscale_factor=upscale_factor)
        if contours:
            # Draw each contour on the overlay in red with a thin line
            cv2.drawContours(overlay, contours, -1, (0, 0, 255), line_thickness)

    # Display the overlay (convert BGR to RGB for matplotlib)
    plt.figure(figsize=(6,6))
    plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    plt.title("Overlay of Isochrones")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    # Replace with the folder containing your screenshots
    images_folder = IMAGE_FOLDER
    overlay_all_isochrones(images_folder, grid_size=33, upscale_factor=10, line_thickness=1)


In [ ]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

def classify_cell_color(avg_bgr):
    """
    Classify a cell color based on the average BGR value.
    Returns one of: 'green', 'orange', or 'white'.
    (Outer border cells will be handled separately as 'black'.)
    """
    b, g, r = avg_bgr
    color_bgr = np.uint8([[avg_bgr]])
    color_hsv = cv2.cvtColor(color_bgr.reshape(1,1,3), cv2.COLOR_BGR2HSV).reshape(3,)
    h, s, v = color_hsv

    # Adjust thresholds as needed
    if 30 <= h <= 90 and s > 50 and v > 50:
        return 'green'
    elif 0 <= h <= 40 and s > 60 and v > 60:
        return 'orange'
    else:
        return 'white'

def process_image_into_grid(img, grid_size=33):
    """
    Splits the image into grid_size x grid_size cells.
    For each cell, computes the average BGR color and
    classifies it as 'black', 'green', 'orange', or 'white'.
    Returns a (grid_size x grid_size) array of labels.
    """
    height, width, _ = img.shape
    cell_h = height // grid_size
    cell_w = width // grid_size

    labels_grid = np.empty((grid_size, grid_size), dtype=object)

    for row in range(grid_size):
        for col in range(grid_size):
            # Force outer border cells to 'black'
            if row == 0 or row == grid_size - 1 or col == 0 or col == grid_size - 1:
                labels_grid[row, col] = 'black'
            else:
                y_start = row * cell_h
                y_end   = (row + 1) * cell_h
                x_start = col * cell_w
                x_end   = (col + 1) * cell_w
                cell = img[y_start:y_end, x_start:x_end]
                avg_b = np.mean(cell[:,:,0])
                avg_g = np.mean(cell[:,:,1])
                avg_r = np.mean(cell[:,:,2])
                labels_grid[row, col] = classify_cell_color((avg_b, avg_g, avg_r))

    return labels_grid

def get_isochrone_contours(labels_grid, upscale_factor=10):
    """
    Creates a binary mask from the classification grid for cells labeled
    'orange' or 'white', upscales it, and returns the contours.
    """
    grid_size = labels_grid.shape[0]
    mask = np.zeros((grid_size, grid_size), dtype=np.uint8)

    # Mark orange/white as 255
    for r in range(grid_size):
        for c in range(grid_size):
            if labels_grid[r, c] in ('orange', 'white'):
                mask[r, c] = 255

    upscaled_size = (grid_size * upscale_factor, grid_size * upscale_factor)
    mask_upscaled = cv2.resize(mask, upscaled_size, interpolation=cv2.INTER_NEAREST)

    contours, _ = cv2.findContours(mask_upscaled, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def overlay_all_isochrones_color_gradient(images_folder, grid_size=33, upscale_factor=10, line_thickness=2):
    """
    For all images in the folder, compute the firefront contours and overlay them
    on a single blank white canvas, using a color gradient to distinguish time steps.
    """
    image_paths = sorted(glob.glob(os.path.join(images_folder, '*.png')), key=os.path.getmtime)
    if not image_paths:
        image_paths = sorted(glob.glob(os.path.join(images_folder, '*.jpg')), key=os.path.getmtime)
    if not image_paths:
        raise ValueError("No images found in the specified folder.")

    # Create a color map with as many distinct colors as there are images
    cmap = cm.get_cmap('jet', len(image_paths))  # or 'viridis', 'plasma', etc.

    upscaled_size = (grid_size * upscale_factor, grid_size * upscale_factor)
    overlay = 255 * np.ones((upscaled_size[1], upscaled_size[0], 3), dtype=np.uint8)  # white canvas

    for i, path in enumerate(image_paths):
        img_bgr = cv2.imread(path)
        if img_bgr is None:
            print(f"Warning: Could not load image {path}")
            continue

        labels_grid = process_image_into_grid(img_bgr, grid_size=grid_size)
        contours = get_isochrone_contours(labels_grid, upscale_factor=upscale_factor)

        # Get a distinct color from the colormap (in RGBA form)
        color_rgba = cmap(i)
        # Convert RGBA -> BGR in [0,255]
        color_bgr = (int(color_rgba[2]*255),
                     int(color_rgba[1]*255),
                     int(color_rgba[0]*255))

        if contours:
            cv2.drawContours(overlay, contours, -1, color_bgr, line_thickness)

    # Display the result
    plt.figure(figsize=(8,8))
    plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    plt.title("Overlay of Isochrones (Color Gradient by Time Step)")
    plt.axis('off')
    plt.show()

# -------------- Example usage --------------
if __name__ == "__main__":
    images_folder = IMAGE_FOLDER
    overlay_all_isochrones_color_gradient(
        images_folder,
        grid_size=33,
        upscale_factor=10,
        line_thickness=1
    )


In [ ]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

def classify_cell_color(avg_bgr):
    b, g, r = avg_bgr
    color_bgr = np.uint8([[avg_bgr]])
    color_hsv = cv2.cvtColor(color_bgr.reshape(1,1,3), cv2.COLOR_BGR2HSV).reshape(3,)
    h, s, v = color_hsv

    if 30 <= h <= 90 and s > 50 and v > 50:
        return 'green'
    elif 0 <= h <= 40 and s > 60 and v > 60:
        return 'orange'
    else:
        return 'white'

def process_image_into_grid(img, grid_size=33):
    height, width, _ = img.shape
    cell_h = height // grid_size
    cell_w = width // grid_size

    labels_grid = np.empty((grid_size, grid_size), dtype=object)

    for row in range(grid_size):
        for col in range(grid_size):
            if row == 0 or row == grid_size - 1 or col == 0 or col == grid_size - 1:
                labels_grid[row, col] = 'black'
            else:
                y_start = row * cell_h
                y_end   = (row + 1) * cell_h
                x_start = col * cell_w
                x_end   = (col + 1) * cell_w
                cell = img[y_start:y_end, x_start:x_end]
                avg_b = np.mean(cell[:,:,0])
                avg_g = np.mean(cell[:,:,1])
                avg_r = np.mean(cell[:,:,2])
                labels_grid[row, col] = classify_cell_color((avg_b, avg_g, avg_r))

    return labels_grid

def get_isochrone_contours(labels_grid, upscale_factor=10):
    grid_size = labels_grid.shape[0]
    mask = np.zeros((grid_size, grid_size), dtype=np.uint8)
    for r in range(grid_size):
        for c in range(grid_size):
            if labels_grid[r, c] in ('orange', 'white'):
                mask[r, c] = 255

    upscaled_size = (grid_size * upscale_factor, grid_size * upscale_factor)
    mask_upscaled = cv2.resize(mask, upscaled_size, interpolation=cv2.INTER_NEAREST)

    contours, _ = cv2.findContours(mask_upscaled, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def alpha_blend_contours(base_image, contours, color_bgr, alpha=0.5, thickness=2):
    """
    Draws the given contours in color_bgr on a separate layer, then
    alpha-blends that layer with the base_image.
    """
    h, w, _ = base_image.shape
    # Create a temporary layer
    contour_layer = base_image.copy()
    cv2.drawContours(contour_layer, contours, -1, color_bgr, thickness)

    # Alpha blend
    blended = cv2.addWeighted(contour_layer, alpha, base_image, 1 - alpha, 0)
    return blended

def overlay_all_isochrones_alpha_blend(images_folder, grid_size=33, upscale_factor=10,
                                       line_thickness=2, alpha=0.5):
    """
    For all images in the folder, compute the firefront contours and overlay them
    on a single blank white canvas using alpha blending. Each time step is drawn
    in a distinct color with partial transparency.
    """
    image_paths = sorted(glob.glob(os.path.join(images_folder, '*.png')), key=os.path.getmtime)
    if not image_paths:
        image_paths = sorted(glob.glob(os.path.join(images_folder, '*.jpg')), key=os.path.getmtime)
    if not image_paths:
        raise ValueError("No images found in the specified folder.")

    cmap = cm.get_cmap('jet', len(image_paths))
    upscaled_size = (grid_size * upscale_factor, grid_size * upscale_factor)
    # Start with a white canvas
    overlay = 255 * np.ones((upscaled_size[1], upscaled_size[0], 3), dtype=np.uint8)

    for i, path in enumerate(image_paths):
        img_bgr = cv2.imread(path)
        if img_bgr is None:
            print(f"Warning: Could not load image {path}")
            continue

        labels_grid = process_image_into_grid(img_bgr, grid_size=grid_size)
        contours = get_isochrone_contours(labels_grid, upscale_factor=upscale_factor)
        if not contours:
            continue

        # Distinct color for each time step
        color_rgba = cmap(i)
        color_bgr = (int(color_rgba[2]*255),
                     int(color_rgba[1]*255),
                     int(color_rgba[0]*255))

        # Alpha blend the contours onto the overlay
        overlay = alpha_blend_contours(overlay, contours, color_bgr,
                                       alpha=alpha, thickness=line_thickness)

    # Display final overlay
    plt.figure(figsize=(8,8))
    plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    plt.title("Overlay of Isochrones (Alpha-Blended)")
    plt.axis('off')
    plt.show()

# -------------- Example usage --------------
if __name__ == "__main__":
    images_folder = IMAGE_FOLDER
    overlay_all_isochrones_alpha_blend(
        images_folder,
        grid_size=33,
        upscale_factor=10,
        line_thickness=1,
        alpha=0.5
    )


In [ ]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

# TIME STEP BETWEEN IMAGES = 1
TIME_STEP = 1  # second

def classify_cell_color(avg_bgr):
    """
    Classify a cell based on its average BGR color.
    Returns:
      'green' for leaves,
      'orange' for fire,
      'white' for burnt/empty.
    """
    b, g, r = avg_bgr
    # Convert BGR to HSV for easier thresholding.
    color_bgr = np.uint8([[avg_bgr]])
    color_hsv = cv2.cvtColor(color_bgr, cv2.COLOR_BGR2HSV)[0, 0]
    h, s, v = color_hsv
    
    # Adjust thresholds as needed:
    if 30 <= h <= 90 and s > 50 and v > 50:
        return 'green'
    elif 0 <= h <= 40 and s > 60 and v > 60:
        return 'orange'
    else:
        return 'white'

def process_image_into_grid(img, grid_size=33):
    """
    Splits the input image into a grid_size x grid_size grid.
    Outer border cells are set to 'black'. Inner cells are classified.
    Returns a (grid_size x grid_size) NumPy array of labels.
    """
    height, width, _ = img.shape
    cell_h = height // grid_size
    cell_w = width // grid_size
    labels = []
    for row in range(grid_size):
        row_labels = []
        for col in range(grid_size):
            if row == 0 or row == grid_size - 1 or col == 0 or col == grid_size - 1:
                row_labels.append('black')
            else:
                y_start = row * cell_h
                y_end   = (row + 1) * cell_h
                x_start = col * cell_w
                x_end   = (col + 1) * cell_w
                cell = img[y_start:y_end, x_start:x_end]
                avg_b = np.mean(cell[:, :, 0])
                avg_g = np.mean(cell[:, :, 1])
                avg_r = np.mean(cell[:, :, 2])
                row_labels.append(classify_cell_color((avg_b, avg_g, avg_r)))
        labels.append(row_labels)
    return np.array(labels)

# Main code to iterate through images, count areas, and plot results.
# Assumes a 1-second time step between images.
# Get all image file paths (using images_folder defined earlier)
image_paths = sorted(glob.glob(os.path.join(images_folder, '*.png')), key=os.path.getmtime)
if not image_paths:
    image_paths = sorted(glob.glob(os.path.join(images_folder, '*.jpg')), key=os.path.getmtime)
if not image_paths:
    raise ValueError("No images found in the specified folder.")

# Lists to store the area (number of cells) per image
fire_reached_area = []  # counts cells that are either 'orange' or 'white'
fire_area = []          # counts cells that are 'orange'

for path in image_paths:
    img = cv2.imread(path)
    if img is None:
        print(f"Warning: Could not load image {path}")
        continue
    grid = process_image_into_grid(img, grid_size=33)
    # Count cells that are either 'orange' or 'white'
    reached_count = np.sum((grid == 'orange') | (grid == 'white'))
    # Count cells that are just 'orange'
    fire_count = np.sum(grid == 'orange')
    fire_reached_area.append(reached_count)
    fire_area.append(fire_count)

# Generate time points (in seconds)
time_points = np.arange(len(fire_reached_area))  # since TIME_STEP is 1, time equals image index

# Print the values that are being plotted:
print("Time (seconds):", time_points)
print("Fire Reached Area (cells, orange+white):", fire_reached_area)
print("Fire Area (cells, orange only):", fire_area)

# Plot the total area over time
plt.figure(figsize=(10,5))
plt.plot(time_points, fire_reached_area, marker='o', label='Fire Reached Area (orange+white)')
plt.plot(time_points, fire_area, marker='s', label='Fire Area (orange)')
plt.xlabel("Time (seconds)")
plt.ylabel("Number of Cells")
plt.title("Fire Spread Area Over Time")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Compute the differences between consecutive images (new cells per second)
fire_reached_diff = np.diff(fire_reached_area)
fire_area_diff = np.diff(fire_area)

# Print the differences (rate of spread)
print("New Fire Reached Area per second (cells):", fire_reached_diff)
print("New Fire Area per second (cells):", fire_area_diff)

# Plot the rate of spread (area increase per second)
plt.figure(figsize=(10,5))
plt.plot(time_points[1:], fire_reached_diff, marker='o', label='New Fire Reached Area per Second')
plt.plot(time_points[1:], fire_area_diff, marker='s', label='New Fire Area per Second')
plt.xlabel("Time (seconds)")
plt.ylabel("New Cells per Second")
plt.title("Fire Spread Rate (Area Increase per Second)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
